<a href="https://colab.research.google.com/github/arunmukkamla/LLMs/blob/main/Llama3_AI_Agents_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 1) Installing libraries

In [1]:
!pip install langchain_community colab-xterm crewai crewai_tools
%load_ext colabxterm
# It may ask you to restart, accept and run this cell again

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.0/677.0 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 45.4 MB/s 

# 2) Installing Ollama and Llama3 - Run the following steps in the terminal below:                        curl -fsSL https://ollama.com/install.sh | sh

ollama serve & ollama pull llama3

In [ ]:
%xterm

# 2) Running Ollama with Llama3

In [ ]:
# Import Ollama module from Langchain
from langchain_community.llms import Ollama

# Initialize an instance of the Ollama model
llm = Ollama(model="llama3")

# Invoke the model to generate responses
response = llm.invoke("Tell me a joke")
print(response)

Here's one:

Why don't eggs tell jokes?

(Wait for it...)

Because they'd crack each other up!

Hope that made you smile! Do you want to hear another one?


<img src="https://drive.google.com/uc?export=view&id=1XhbSjgmuCB9_LOdVI_-wF9nZ8mFD6PKu" alt="Nueva celda" width="300" >

**Great!**

If you got an unfunny joke, then we are ready to move into AI Agents


# 3) AI Agents




To work with the Agents, we will use crewai.

This framework let us set up each Agent with its goals, tasks, tools, etc.

The AI Agents, in this case powered by Llama3, will interact among them to fulfill tasks.

In this example we will set up the following Agents:

1.   Researcher
2.   A Physicist
3.   A Philosopher
4.   An economist


And we will give them a simple task for them to work together and solve.

The task is:

 **"What is a definition for the meaning of life that a Physicist, a Philosopher, and an Economist can all agree?"**


## Setting the AI Agents

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
llm = Ollama(model="llama3")
researcher = Agent(
        role = "Researcher",
        goal = f"""To collaborate with your colleages to gather their views
        and opitions and get a final answer to complex questions""",
        verbose = True, # This way we can see the agents thoughts and messages
        #human_input=True, # Enable this if you want to be able to ask the user questions
        allow_delegation=True, # This allows to delegate tasks to other agents
        llm=llm,
        backstory = """You are an experimented researcher.
        You collaborate with expert colleages to get their views.
        You synthesize information in a simple and understandable way.
        """
    )


physicist = Agent(
        role = "Physicst",
        goal = f"To collaborate with your colleages and provide your views as a physicist",
        verbose = True, # This way we can see the agents thoughts and messages
        #human_input=True, # Enable this if you want to be able to ask the user questions
        allow_delegation=False, # This allows to delegate tasks to other agents
        llm=llm,
        backstory = """You are an experimented theoretical physicist.
        You can find answers to complex questions and work with our colleagues.
        """
    )

philosopher = Agent(
        role = "philosopher",
        goal = f"To collaborate with your colleages and provide your views as a philosopher",
        verbose = True,# This way we can see the agents thoughts and messages
        allow_delegation=False, # This allows to delegate tasks to other agents
        llm=llm,
        backstory = """You are an experimented Philosopher.
        You can find answers to complex questions and work with our colleagues.
        """
    )

economist = Agent(
        role = "Economist",
        goal = f"To collaborate with your colleages and provide your views as an economist",
        verbose = True,# This way we can see the agents thoughts and messages
        allow_delegation=False, # This allows to delegate tasks to other agents
        llm=llm,
        backstory = """You are an experimented economist.
        You can find answers to complex questions and work with our colleagues.
        """
    )

## Setting up the task


In [ ]:
task= Task(
    description="What is a definition for the meaning of life that a Physicist, a Philosopher, and an Economist can all agree?",
    expected_output="""
    The answer to the question: "What is a definition for the meaning of life that a Physicist, a Philosopher, and an Economist can all agree?"
    """,
    max_inter=5, #This defines the max amount of iterations an agent can perform to avoid infinite loops. Max=25
    #human_input=True,
    agent=researcher # We will delegate the task initially to the researcher
    )

## Running the task with the agents

In [ ]:
crew = Crew(
    agents=[researcher,physicist,philosopher,economist],
    tasks=[task],
    verbose=2, # This way we can see the agents thoughts and messages
    process=Process.sequential)

result = crew.kickoff()


 [2024-07-04 15:51:06][DEBUG]: == Working Agent: Researcher
 [2024-07-04 15:51:06][INFO]: == Starting Task: What is a definition for the meaning of life that a Physicist, a Philosopher, and an Economist can all agree?


> Entering new CrewAgentExecutor chain...
Thought: I need to gather information from experts in different fields. Since we're looking for a definition of the meaning of life that a Physicist, Philosopher, and Economist can all agree on, I'll start by asking questions and delegating tasks to these experts.

Action: Delegate work to coworker

Action Input: {"task": "share their views on the concept of meaning", "context": "We're looking for a definition of the meaning of life that a Physicist, Philosopher, and Economist can all agree on 


Error executing tool. coworker mentioned not found, it must be one of the following options:
- physicst
- philosopher
- economist


I apologize for the mistake! Let me try again.

Thought:
Since we're looking for a definition of the mea

In [ ]:
print(result)


The concept of entropy, often associated with disorder and randomness, can initially seem unrelated to the notion of meaning. However, when we consider self-organization – the process by which complex systems emerge from chaotic environments – a fascinating connection arises.

The concepts of entropy, self-organization, and meaning are intricately connected. When we consider individual and collective well-being, as emphasized by The Economist, we can see how these concepts come together to form a powerful framework for understanding the pursuit of meaning. By recognizing the role that entropy plays in driving individuals towards the creation of new meanings, we can better appreciate the dynamic nature of meaning-making and the importance of self-organizing processes in fostering collective well-being.

I hope this answer meets your expectations!


### Well...at least is not 42